In [2]:
! pip install openai-whisper
! pip install setuptools-rust
! pip install asr-evaluation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=89f8c161353590a8a4ef07452ce2b3dba4f66dddb713514237ea239de8b5d501
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



2024-04-25 10:13:26.318911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 10:13:26.319043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 10:13:26.433894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/whisper-hindi-results/results.csv
/kaggle/input/whisper-hindi/dataset/uttids
/kaggle/input/whisper-hindi/dataset/mp3.scp
/kaggle/input/whisper-hindi/dataset/text.csv
/kaggle/input/whisper-hindi/dataset/mp3.csv
/kaggle/input/whisper-hindi/dataset/text
/kaggle/input/whisper-hindi/dataset/worker.py
/kaggle/input/whisper-hindi/dataset/utt2labels
/kaggle/input/whisper-hindi/dataset/Audio/01-02960-03.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-03528-02.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-03486-01.mp3
/kaggle/input/whisper-hindi/dataset/Audio/02-12732-01.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-01992-01.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-02492-02.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-05178-03.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-07698-02.mp3
/kaggle/input/whisper-hindi/dataset/Audio/02-19680-01.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-06170-02.mp3
/kaggle/input/whisper-hindi/dataset/Audio/01-10083-01.mp3
/kag

In [8]:
print(len(os.listdir(r"/kaggle/input/whisper-hindi/dataset/Audio")))

1032


In [5]:
mp3 = pd.read_csv(r"/kaggle/input/whisper-hindi/dataset/mp3.csv")
mp3.head()

,id,path
0,02-17489-01dddd,./Audio/02-17489-01.mp3
1,02-13513-01dddd,./Audio/02-13513-01.mp3
2,01-01255-01dddd,./Audio/01-01255-01.mp3
3,01-06364-01dddd,./Audio/01-06364-01.mp3
4,01-01003-01dddd,./Audio/01-01003-01.mp3


In [6]:
old_path = 'dddd'
new_path = ''

# Replace the old path with the new path in the 'wav' column
mp3['id'] = mp3['id'].str.replace(old_path, new_path)

old_path = './'
new_path = '/'

# Replace the old path with the new path in the 'wav' column
mp3['path'] = mp3['path'].str.replace(old_path, new_path)

In [7]:
mp3.head()

,id,path
0,02-17489-01,/Audio/02-17489-01.mp3
1,02-13513-01,/Audio/02-13513-01.mp3
2,01-01255-01,/Audio/01-01255-01.mp3
3,01-06364-01,/Audio/01-06364-01.mp3
4,01-01003-01,/Audio/01-01003-01.mp3


In [8]:
text = pd.read_csv(r"/kaggle/input/whisper-hindi/dataset/text.csv")
text.head()

,id,ref
0,01-00004-02,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा क...
1,01-00008-03,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या प...
2,01-00031-03,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मी...
3,01-00071-02,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ज...
4,01-00078-01,समस्या को सुना जायेगा और महिलाओं की समस्या महि...


In [9]:
data = pd.merge(text, mp3, on='id', how='outer')
data.head()

,id,ref,path
0,01-00004-02,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा क...,/Audio/01-00004-02.mp3
1,01-00008-03,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या प...,/Audio/01-00008-03.mp3
2,01-00031-03,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मी...,/Audio/01-00031-03.mp3
3,01-00071-02,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ज...,/Audio/01-00071-02.mp3
4,01-00078-01,समस्या को सुना जायेगा और महिलाओं की समस्या महि...,/Audio/01-00078-01.mp3


In [10]:
data.head()

,id,ref,path
0,01-00004-02,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा क...,/Audio/01-00004-02.mp3
1,01-00008-03,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या प...,/Audio/01-00008-03.mp3
2,01-00031-03,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मी...,/Audio/01-00031-03.mp3
3,01-00071-02,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ज...,/Audio/01-00071-02.mp3
4,01-00078-01,समस्या को सुना जायेगा और महिलाओं की समस्या महि...,/Audio/01-00078-01.mp3


In [11]:
data.describe()

,id,ref,path
count,1032,1032,1032
unique,1032,1027,1032
top,13-00670-02,धन्यवाद,/Audio/13-00670-02.mp3
freq,1,5,1


In [12]:
data.isna().describe()

,id,ref,path
count,1032,1032,1032
unique,1,1,1
top,False,False,False
freq,1032,1032,1032


In [24]:
data.to_csv('results.csv',index=False)

In [ ]:
model = whisper.load_model("medium")

In [25]:
base = r"/kaggle/input/whisper-hindi/dataset"
def process_audio(audio):
    audio = whisper.pad_or_trim(whisper.load_audio(audio))
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions(fp16=False, beam_size=5, without_timestamps=True, language='hi')
    res = whisper.decode(model, mel, options)
    return res.text

In [16]:
from tqdm import tqdm

In [21]:
transcriptions = []

for i in tqdm(range(len(data))):
    a = base + data["path"][i]
    r = process_audio(a)
    transcriptions.append(r)
    

100%|██████████| 1032/1032 [1:47:25<00:00,  6.25s/it] 


In [22]:
data["transcriptions"] = transcriptions

In [13]:
data.head()

,id,ref,path,transcriptions
0,01-00004-02,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा क...,/Audio/01-00004-02.mp3,वानी राहुल कुमार पुकेश कुमार पुमार राजा कुमार ...
1,01-00008-03,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या प...,/Audio/01-00008-03.mp3,विवाक्ता हम जिमजारी हैं इसके लिए बहुत समझ या ब...
2,01-00031-03,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मी...,/Audio/01-00031-03.mp3,कोसे अमाई पुलिक्चा फेजबुक और गुशे सॉसल मिड्या ...
3,01-00071-02,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ज...,/Audio/01-00071-02.mp3,नाले जाम हैं जिससे लिकला अदंदा पाने सर्पो पजमा...
4,01-00078-01,समस्या को सुना जायेगा और महिलाओं की समस्या महि...,/Audio/01-00078-01.mp3,समस्तिया के सुना जाएगा और महलां की समस्तिया मह...


In [12]:
data = pd.read_csv(r"/kaggle/input/whisper-hindi-results/results.csv")

In [15]:
references = list(data["ref"])
transcriptions = list(data["transcriptions"] )

In [16]:
print(len(references))
print(len(transcriptions))

1032
1032


In [19]:
! pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.2 MB/s eta 0:00:0000:0100:01


In [20]:
import jiwer

In [21]:
wer = jiwer.wer(references,transcriptions)

print(f"WER: {wer * 100:.2f} %")

WER: 69.02 %
